In [1]:
import numpy as np
from hyperopt import hp, fmin, tpe
from functions import CreatDataGBM
import xgboost as xgb

In [2]:
X_train,Y_train,X_val,Y_val,Weight = CreatDataGBM()

/home/will/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
def NWRMSLE_exp(preds, dtrain):
    labels = dtrain.get_label()
    preds = np.exp(preds)
    temp1 = preds/(preds+1)
    temp2 = np.log(preds+1) - labels
    return temp2*temp1, temp1*temp1 + temp2*temp1*(1-temp1)

In [4]:
def NWRMSLE_relu(preds, dtrain):
    labels = dtrain.get_label()
    preds2 = np.maximum(preds,0)
    inc = preds>0
    temp1 = inc/(1+preds2)
    temp2 = np.log(preds2+1) - labels
    sq = temp1*temp1
    return temp2*temp1, sq - temp2*sq

In [5]:
'''def NWRMSLE_sq(preds, dtrain):
    labels = dtrain.get_label()
    preds2 = preds**2
    temp1 = preds/(1+preds2)
    temp2 = np.log(preds2+1) - labels
    return temp2*temp1, temp1*temp1 + temp2*(3*preds2+1)/(preds2+1)**2'''

'def NWRMSLE_sq(preds, dtrain):\n    labels = dtrain.get_label()\n    preds2 = preds**2\n    temp1 = preds/(1+preds2)\n    temp2 = np.log(preds2+1) - labels\n    return temp2*temp1, temp1*temp1 + temp2*(3*preds2+1)/(preds2+1)**2'

In [6]:
objective_fun = [(NWRMSLE_exp,np.exp),(NWRMSLE_relu,lambda x:np.maximum(x,0))]

In [7]:
def loss_func(Weight,yhat,y):
    return np.sqrt(np.sum(Weight*(np.log(yhat+1)-y)**2)/np.sum(Weight))

In [8]:
param = {}
param['eta'] = 0.25
param['max_depth'] = 10
param['min_child_weight'] = 1
param['tree_method'] = 'hist'
monitor = 20
monitorTimes = 7

In [16]:
def hyperSearchGBM(paras):   
    model_para = param.copy()
    model_para['subsample'] = paras[0]
    model_para['colsample_bytree'] = paras[1]
    model_para['gamma'] = paras[2]
    obj,link = objective_fun[paras[3]]
    tot_loss = 0
    for i in range(16):
        dtrain = xgb.DMatrix(
            X_train, label=Y_train[:, i],
            weight=np.tile(Weight,16)
        )
        dval = xgb.DMatrix(
            X_val, label=Y_val[:, i],
            weight=Weight)
        
        best_loss = 100
        for j in range(monitorTimes):           
            # train
            if j == 0:
                model = xgb.train(model_para, dtrain, monitor,obj=obj)
            else:
                model = xgb.train(model_para, dtrain, monitor,obj=NWRMSLE_relu,xgb_model=model)
            # eval
            yhat = link(model.predict(dval,output_margin=True))
            loss = loss_func(Weight,yhat,Y_val[:, i])
            if loss < best_loss:
                best_loss = loss
            else:
                break # early stop
        tot_loss = tot_loss + best_loss

    tot_loss = tot_loss/16    
    print "loss:{} , subsample:{}, colsample:{}, gamma:{} ,objective:{} \n"\
              .format(tot_loss,paras[0],paras[1],paras[2],paras[3])  
    return tot_loss

In [17]:
space = [hp.uniform('subsample',0.01,0.15),\
         hp.uniform('colsample',0.1,0.8),\
         hp.uniform('gamma',1,100),\
         hp.choice('models',[0,1])]

In [18]:
best = fmin(hyperSearchGBM,space,tpe.suggest,60)

loss:0.449536966995 , subsample:0.0550761473573, colsample:0.645103497042, gamma:51.3732251887 ,objective:0 

loss:0.451796226477 , subsample:0.138429686425, colsample:0.501925709187, gamma:1.9867476064 ,objective:0 

loss:0.431957986459 , subsample:0.0556206950421, colsample:0.526688633213, gamma:10.3156374802 ,objective:1 

loss:0.473952657583 , subsample:0.0416467203721, colsample:0.394301144392, gamma:50.3403500953 ,objective:0 

loss:0.4412054121 , subsample:0.139355950746, colsample:0.172517980542, gamma:23.0057283308 ,objective:1 

loss:0.440183298222 , subsample:0.0945076372895, colsample:0.291004901366, gamma:72.744758734 ,objective:1 

loss:0.43047335466 , subsample:0.0721617898053, colsample:0.766616468485, gamma:28.7265834674 ,objective:1 

loss:0.463586819607 , subsample:0.110491032977, colsample:0.329793243217, gamma:29.0351299682 ,objective:0 

loss:0.459741106811 , subsample:0.0430870172236, colsample:0.534802592304, gamma:29.0052059014 ,objective:0 

loss:0.45217672644

Determine number of iterations for smaller learning rate

In [32]:
param = {}
param['eta'] = 0.05
param['max_depth'] = 10
param['min_child_weight'] = 1
param['tree_method'] = 'hist'
monitor = 100
monitorTimes = 10
best_numberStep_list = []
Y_ensemble = np.zeros((yhat.shape[0],16))

In [33]:
model_para = param.copy()
model_para['subsample'] = 0.133448450345
model_para['colsample_bytree'] = 0.367127373623
model_para['gamma'] = 5.18910389561
obj,link = objective_fun[1]
tot_loss = 0

for i in range(16):
    dtrain = xgb.DMatrix(
        X_train, label=Y_train[:, i],
        weight=np.tile(Weight,16)
    )
    dval = xgb.DMatrix(
        X_val, label=Y_val[:, i],
        weight=Weight)

    best_loss = 100
    best_step = 0
    for j in range(monitorTimes):           
        # train
        if j == 0:
            model = xgb.train(model_para, dtrain, monitor,obj=obj)
        else:
            model = xgb.train(model_para, dtrain, monitor,obj=obj,xgb_model=model)
        # eval
        yhat = link(model.predict(dval,output_margin=True))
        loss = loss_func(Weight,yhat,Y_val[:, i])
        print "model:{}, iteration:{}, loss:{}".format(i,(j+1)*monitor,loss)
        if loss < best_loss:
            best_loss = loss
            best_step = (j+1)*monitor
            Y_ensemble[:,i] = yhat
        else:
            break # early stop
    best_numberStep_list.append(best_step)
    tot_loss = tot_loss + best_loss

tot_loss = tot_loss/16    

model:0, iteration:100, loss:0.448519396937
model:0, iteration:200, loss:0.409481455129
model:0, iteration:300, loss:0.404412220765
model:0, iteration:400, loss:0.403457361237
model:0, iteration:500, loss:0.403002472531
model:0, iteration:600, loss:0.402613365348
model:0, iteration:700, loss:0.402369634941
model:0, iteration:800, loss:0.402210586201
model:0, iteration:900, loss:0.401973402677
model:0, iteration:1000, loss:0.401864974931
model:1, iteration:100, loss:0.507223579626
model:1, iteration:200, loss:0.447702234991
model:1, iteration:300, loss:0.440161349527
model:1, iteration:400, loss:0.437287669192
model:1, iteration:500, loss:0.436030312213
model:1, iteration:600, loss:0.435662906922
model:1, iteration:700, loss:0.435279301057
model:1, iteration:800, loss:0.43503370697
model:1, iteration:900, loss:0.434904704589
model:1, iteration:1000, loss:0.434914598316
model:2, iteration:100, loss:0.481579413111
model:2, iteration:200, loss:0.429252650796
model:2, iteration:300, loss:0.

In [34]:
best_numberStep_list

[1000,
 900,
 500,
 200,
 200,
 500,
 1000,
 1000,
 1000,
 300,
 600,
 1000,
 200,
 900,
 700,
 1000]

In [39]:
best_numberStep_list2 = []
Y_ensemble2 = np.zeros((yhat.shape[0],16))

In [40]:
model_para2 = param.copy()
model_para2['subsample'] = 0.119398299563
model_para2['colsample_bytree'] = 0.367823795823
model_para2['gamma'] = 4.43745570732
obj,link = objective_fun[1]
tot_loss = 0

for i in range(16):
    dtrain = xgb.DMatrix(
        X_train, label=Y_train[:, i],
        weight=np.tile(Weight,16)
    )
    dval = xgb.DMatrix(
        X_val, label=Y_val[:, i],
        weight=Weight)

    best_loss = 100
    best_step = 0
    for j in range(monitorTimes):           
        # train
        if j == 0:
            model = xgb.train(model_para2, dtrain, monitor,obj=obj)
        else:
            model = xgb.train(model_para2, dtrain, monitor,obj=NWRMSLE_relu,xgb_model=model)
        # eval
        yhat = link(model.predict(dval,output_margin=True))
        loss = loss_func(Weight,yhat,Y_val[:, i])
        print "model:{}, iteration:{}, loss:{}".format(i,(j+1)*monitor,loss)
        if loss < best_loss:
            best_loss = loss
            best_step = (j+1)*monitor
            Y_ensemble2[:,i] = yhat
        else:
            break # early stop
    best_numberStep_list2.append(best_step)
    tot_loss = tot_loss + best_loss

tot_loss = tot_loss/16    

model:0, iteration:100, loss:0.458784031928
model:0, iteration:200, loss:0.412901957087
model:0, iteration:300, loss:0.405807078045
model:0, iteration:400, loss:0.404271653414
model:0, iteration:500, loss:0.403304264521
model:0, iteration:600, loss:0.403040903899
model:0, iteration:700, loss:0.40274452091
model:0, iteration:800, loss:0.402561174964
model:0, iteration:900, loss:0.402463396366
model:0, iteration:1000, loss:0.402198341158
model:1, iteration:100, loss:0.502047664898
model:1, iteration:200, loss:0.446990787329
model:1, iteration:300, loss:0.440590382684
model:1, iteration:400, loss:0.43784292808
model:1, iteration:500, loss:0.436700371386
model:1, iteration:600, loss:0.436034819886
model:1, iteration:700, loss:0.436550931503
model:2, iteration:100, loss:0.484989600709
model:2, iteration:200, loss:0.429002728707
model:2, iteration:300, loss:0.422527021203
model:2, iteration:400, loss:0.421637045292
model:2, iteration:500, loss:0.421140377911
model:2, iteration:600, loss:0.42

In [41]:
best_numberStep_list2

[1000,
 600,
 1000,
 600,
 200,
 1000,
 1000,
 1000,
 1000,
 1000,
 500,
 1000,
 1000,
 700,
 600,
 1000]

In [42]:
tot_loss

0.41915811139422277

In [45]:
for i in range(16):
    print "model1:{}, model2:{}, combined:{}"\
    .format(loss_func(Weight,Y_ensemble[:,i],Y_val[:, i]),\
            loss_func(Weight,Y_ensemble2[:,i],Y_val[:, i]),\
            loss_func(Weight,(Y_ensemble[:,i]+Y_ensemble2[:,i])/2,Y_val[:, i]))

model1:0.401864974725, model2:0.402198341154, combined:0.401819730031
model1:0.43490470465, model2:0.43603481987, combined:0.435245132073
model1:0.421205497832, model2:0.420560307968, combined:0.420559738947
model1:0.417062048735, model2:0.422263442743, combined:0.418423464793
model1:0.434229300959, model2:0.435695713566, combined:0.434696929222
model1:0.420161125373, model2:0.421505558291, combined:0.420570734608
model1:0.428430376285, model2:0.428263634707, combined:0.428058013732
model1:0.410468745436, model2:0.410918017332, combined:0.410457298723
model1:0.406251438767, model2:0.40589026925, combined:0.405850052854
model1:0.410406806262, model2:0.40929488191, combined:0.409215754369
model1:0.421440510143, model2:0.420863995878, combined:0.420969839292
model1:0.433445681606, model2:0.432303967905, combined:0.432621998666
model1:0.432367104447, model2:0.428608464103, combined:0.42912545019
model1:0.421992559822, model2:0.422486625891, combined:0.422049304686
model1:0.405834319093, mo